In [1]:
import torch
def cosine_scheduler(optimizer, epochs, warmup=3):
    """cosine annealing lr scheduler"""
    return torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=epochs - warmup,
        eta_min=1e-6
    )

In [2]:
import torch
import torch.nn as nn

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction="mean"):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.bce = nn.BCEWithLogitsLoss(reduction="none")

    def forward(self, preds, targets):
        bce_loss = self.bce(preds, targets)
        pt = torch.exp(-bce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * bce_loss
        if self.reduction == "mean":
            return focal_loss.mean()
        elif self.reduction == "sum":
            return focal_loss.sum()
        else:
            return focal_loss


In [6]:
import os
import numpy as np
import yaml
import torch
from ultralytics import YOLO

# --------------------------
# Функция подсчета объектов по классам
# --------------------------
def count_classes(data_yaml: str):
    with open(data_yaml, "r") as f:
        data = yaml.safe_load(f)

    names = data["names"]
    num_classes = len(names)

    root = data["path"]
    label_dirs = [
        os.path.join(root, "labels", "train"),
        os.path.join(root, "labels", "val")
    ]

    counts = np.zeros(num_classes, dtype=np.int64)

    for label_dir in label_dirs:
        if not os.path.exists(label_dir):
            continue
        for file in os.listdir(label_dir):
            if file.endswith(".txt"):
                path = os.path.join(label_dir, file)
                with open(path, "r") as f:
                    for line in f:
                        if line.strip() == "":
                            continue
                        cls_id = int(line.split()[0])
                        counts[cls_id] += 1

    return names, counts

# --------------------------
# Загружаем датасет и считаем alpha
# --------------------------
data_yaml = "dataset.yaml" 
names, counts = count_classes(data_yaml)
print("Классы:", names)
print("Количество объектов по классам:", counts.tolist())

# Построим alpha вектор для FocalLoss
inv = 1.0 / (counts + 1e-6)
alpha = inv / inv.sum()
alpha = torch.tensor(alpha, dtype=torch.float32)
print("Alpha вектор:", alpha.tolist())

# --------------------------
# Инициализация модели
# --------------------------
model = YOLO("yolov8n.pt")

# --------------------------
# Кастомный FocalLoss
# --------------------------
focal_loss = FocalLoss(alpha=alpha, gamma=2.0)

def custom_loss(preds, targets):
    # стандартный лосс для bbox/obj/dfl
    loss_items = model.trainer.criterion(preds, targets)

    # классификация
    cls_preds = preds[0].cls
    targets_dict = model.trainer.build_targets(preds, targets)[0]
    cls_targets = targets_dict["cls"].to(cls_preds.device)

    focal_cls_loss = focal_loss(cls_preds, cls_targets)

    # финальный лосс
    total_loss = (
        loss_items["box"] +
        loss_items["obj"] +
        loss_items.get("dfl", 0) +
        focal_cls_loss
    )
    return total_loss

# --------------------------
# Callback для подмены loss
# --------------------------
def on_train_start(trainer):
    trainer.loss = custom_loss

model.add_callback("on_train_start", on_train_start)



Классы: {0: 'Adjustable_wrench', 1: 'screwdriver_1', 2: 'screwdriver_2', 3: 'Offset_Phillips_screwdriver', 4: 'Side_cutters', 5: 'Shernica', 6: 'Safety_pliers', 7: 'Pliers', 8: 'Rotary_wheel', 9: 'Open_end_wrench', 10: 'Oil_can_opener'}
Количество объектов по классам: [715, 240, 396, 349, 335, 735, 944, 407, 836, 449, 297]
Alpha вектор: [0.05461550131440163, 0.16270866990089417, 0.09861131757497787, 0.11189135164022446, 0.11656741052865982, 0.05312936380505562, 0.041366614401340485, 0.09594614803791046, 0.046710625290870667, 0.0869712308049202, 0.13148175179958344]
Transferred 319/355 items from pretrained weights


In [7]:
# --------------------------
# Тренировка
# --------------------------

results = model.train(
    data=data_yaml,
    epochs=200,
    imgsz=640,
    device="cuda",
    batch=20,
    optimizer="AdamW",
    lr0=0.00044372638023533847,
    weight_decay=0.0015776452546715513,
    momentum=0.8792977803712093,
)

New https://pypi.org/project/ultralytics/8.3.204 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.203  Python-3.12.6 torch-2.10.0.dev20250926+cu130 CUDA:0 (NVIDIA GeForce RTX 5070, 12227MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=20, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00044372638023533847, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.8792977803712093, mosaic=1.0, multi_scale=False, nam